# 심장 질환 데이터 (heart disease)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
df = pd.read_csv('heart_disease.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [3]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [4]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [5]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')

In [6]:
from sklearn.model_selection import cross_val_score
import numpy as np
scores = cross_val_score(model, X, y, cv=5)
print('정확도:', np.round(scores, 2))
print('정확도 평균: %0.2f' % (scores.mean()))

정확도: [0.85 0.85 0.77 0.78 0.77]
정확도 평균: 0.81


In [7]:
from sklearn.model_selection import StratifiedKFold

In [8]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [9]:
print('전체 데이터의 클래스 분포:', np.bincount(y))
print()
for split_no, (train_idx, test_idx) in enumerate(kfold.split(X, y)):
    print('{}번째 훈련 폴드:'.format(split_no+1), np.bincount(y[train_idx]))
    print('{}번째 검증 폴드:'.format(split_no+1), np.bincount(y[test_idx]))
    print()

전체 데이터의 클래스 분포: [138 165]

1번째 훈련 폴드: [110 132]
1번째 검증 폴드: [28 33]

2번째 훈련 폴드: [110 132]
2번째 검증 폴드: [28 33]

3번째 훈련 폴드: [110 132]
3번째 검증 폴드: [28 33]

4번째 훈련 폴드: [111 132]
4번째 검증 폴드: [27 33]

5번째 훈련 폴드: [111 132]
5번째 검증 폴드: [27 33]



In [10]:
scores = cross_val_score(model, X, y, cv=kfold)
print('정확도:', np.round(scores, 2))
print('정확도 평균: %0.2f' % (scores.mean()))

정확도: [0.75 0.75 0.75 0.82 0.87]
정확도 평균: 0.79


In [11]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)

In [12]:
def grid_search(params, random=False): 

    xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', 
                        random_state=2, verbosity=0, use_label_encoder=False)  
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)  
    if random:
        grid = RandomizedSearchCV(xgb, params, cv=kfold, n_iter=20, 
                                  n_jobs=-1, random_state=2)
    else:
        grid = GridSearchCV(xgb, params, cv=kfold, n_jobs=-1)
    grid.fit(X, y)
    best_params = grid.best_params_
    print("최상의 매개변수:", best_params)
    best_score = grid.best_score_
    print("최상의 점수: {:.5f}".format(best_score))

In [13]:
grid_search(params={'n_estimators':[100, 200, 400, 800]})

최상의 매개변수: {'n_estimators': 100}
최상의 점수: 0.78913


In [14]:
grid_search(params={'learning_rate':[0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]})

최상의 매개변수: {'learning_rate': 0.05}
최상의 점수: 0.80563


In [15]:
grid_search(params={'max_depth':[2, 3, 5, 6, 8]})

최상의 매개변수: {'max_depth': 2}
최상의 점수: 0.80568


In [16]:
grid_search(params={'gamma':[0, 0.01, 0.1, 0.5, 1, 2]})

최상의 매개변수: {'gamma': 1}
최상의 점수: 0.80224


In [17]:
grid_search(params={'min_child_weight':[1, 2, 3, 4, 5]})

최상의 매개변수: {'min_child_weight': 2}
최상의 점수: 0.80880


In [18]:
grid_search(params={'subsample':[0.5, 0.7, 0.8, 0.0, 1]})

최상의 매개변수: {'subsample': 0.7}
최상의 점수: 0.80568


In [19]:
grid_search(params={'colsample_bytree': [0.5, 0.7, 0.8, 0.9, 1]})

최상의 매개변수: {'colsample_bytree': 0.5}
최상의 점수: 0.79891


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [21]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')

In [22]:
eval_set = [(X_test, y_test)]

In [23]:
eval_metric = 'error'

In [24]:
model.fit(X_train, y_train, eval_metric=eval_metric,
          eval_set=eval_set)

[0]	validation_0-error:0.171053
[1]	validation_0-error:0.118421
[2]	validation_0-error:0.157895
[3]	validation_0-error:0.184211
[4]	validation_0-error:0.157895
[5]	validation_0-error:0.171053
[6]	validation_0-error:0.157895
[7]	validation_0-error:0.144737
[8]	validation_0-error:0.144737
[9]	validation_0-error:0.144737
[10]	validation_0-error:0.131579
[11]	validation_0-error:0.144737
[12]	validation_0-error:0.144737
[13]	validation_0-error:0.144737
[14]	validation_0-error:0.144737
[15]	validation_0-error:0.144737
[16]	validation_0-error:0.157895
[17]	validation_0-error:0.131579
[18]	validation_0-error:0.144737
[19]	validation_0-error:0.144737
[20]	validation_0-error:0.144737
[21]	validation_0-error:0.144737
[22]	validation_0-error:0.144737
[23]	validation_0-error:0.144737
[24]	validation_0-error:0.144737
[25]	validation_0-error:0.144737
[26]	validation_0-error:0.131579
[27]	validation_0-error:0.131579
[28]	validation_0-error:0.131579
[29]	validation_0-error:0.131579
[30]	validation_0-er

XGBClassifier()

In [25]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("정확도: %.2f%%" % (accuracy * 100.0))

정확도: 85.53%


In [26]:
model = XGBClassifier(booster='gbtree', objective='binary:logistic')
eval_set = [(X_test, y_test)]
eval_metric = "error"
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set,
          early_stopping_rounds=10, verbose=True)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("정확도: %.2f%%" % (accuracy * 100.0))

[0]	validation_0-error:0.171053
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.118421
[2]	validation_0-error:0.157895
[3]	validation_0-error:0.184211
[4]	validation_0-error:0.157895
[5]	validation_0-error:0.171053
[6]	validation_0-error:0.157895
[7]	validation_0-error:0.144737
[8]	validation_0-error:0.144737
[9]	validation_0-error:0.144737
[10]	validation_0-error:0.131579
[11]	validation_0-error:0.144737
Stopping. Best iteration:
[1]	validation_0-error:0.118421

정확도: 88.16%


In [27]:
model = XGBClassifier(n_estimators=5000)
eval_set = [(X_test, y_test)]
eval_metric = "error"
model.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set,
          early_stopping_rounds=100)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("정확도: %.2f%%" % (accuracy * 100.0))

[0]	validation_0-error:0.171053
Will train until validation_0-error hasn't improved in 100 rounds.
[1]	validation_0-error:0.118421
[2]	validation_0-error:0.157895
[3]	validation_0-error:0.184211
[4]	validation_0-error:0.157895
[5]	validation_0-error:0.171053
[6]	validation_0-error:0.157895
[7]	validation_0-error:0.144737
[8]	validation_0-error:0.144737
[9]	validation_0-error:0.144737
[10]	validation_0-error:0.131579
[11]	validation_0-error:0.144737
[12]	validation_0-error:0.144737
[13]	validation_0-error:0.144737
[14]	validation_0-error:0.144737
[15]	validation_0-error:0.144737
[16]	validation_0-error:0.157895
[17]	validation_0-error:0.131579
[18]	validation_0-error:0.144737
[19]	validation_0-error:0.144737
[20]	validation_0-error:0.144737
[21]	validation_0-error:0.144737
[22]	validation_0-error:0.144737
[23]	validation_0-error:0.144737
[24]	validation_0-error:0.144737
[25]	validation_0-error:0.144737
[26]	validation_0-error:0.131579
[27]	validation_0-error:0.131579
[28]	validation_0-e

In [28]:
grid_search(params={'n_estimators':[2, 25, 50, 75, 100]})

최상의 매개변수: {'n_estimators': 75}
최상의 점수: 0.80230


In [29]:
grid_search(params={'max_depth': [1, 2, 3, 4, 6, 7, 8],
                    'n_estimators':[50]})

최상의 매개변수: {'max_depth': 2, 'n_estimators': 50}
최상의 점수: 0.81230


In [30]:
grid_search(params={'max_depth': [1, 2, 3, 4, 6, 7, 8],
                    'n_estimators':[2, 50, 100]})

최상의 매개변수: {'max_depth': 1, 'n_estimators': 100}
최상의 점수: 0.82546


In [31]:
grid_search(params={'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'learning_rate': 0.3, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.83869


In [32]:
grid_search(params={'min_child_weight': [1, 2, 3, 4, 5],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 50}
최상의 점수: 0.80902


In [34]:
grid_search(params={'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'max_depth': 1, 'n_estimators': 50, 'subsample': 0.8}
최상의 점수: 0.82536


In [35]:
grid_search(params={'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'min_child_weight': [1, 2, 3, 4, 5],
                    'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
                    'max_depth': [1, 2, 3, 4, 5],
                    'n_estimators': [2]})

최상의 매개변수: {'learning_rate': 0.5, 'max_depth': 2, 'min_child_weight': 4, 'n_estimators': 2, 'subsample': 0.9}
최상의 점수: 0.81224


In [36]:
grid_search(params={'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'min_child_weight': [1, 2, 3, 4, 5],
                    'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5],
                    'max_depth': [1, 2, 3, 4, 5, None],
                    'n_estimators': [2, 25, 50, 75, 100]}, random=True)

최상의 매개변수: {'subsample': 0.6, 'n_estimators': 25, 'min_child_weight': 4, 'max_depth': 4, 'learning_rate': 0.5}
최상의 점수: 0.82208


In [37]:
grid_search(params={'colsample_bytree':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'colsample_bytree': 0.5, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.81557


In [38]:
grid_search(params={'colsample_bylevel':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'colsample_bylevel': 0.5, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.81557


In [39]:
grid_search(params={'colsample_bynode':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'colsample_bylevel':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'colsample_bytree':[0.5, 0.6, 0.7, 0.8, 0.9, 1],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'colsample_bylevel': 0.5, 'colsample_bynode': 0.5, 'colsample_bytree': 0.9, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.84191


In [40]:
grid_search(params={'gamma': [0, 0.01, 0.05, 0.1, 0.5, 1, 2, 3],
                    'colsample_bylevel':[0.9],
                    'colsample_bytree':[0.8],
                    'colsample_bynode': [0.5],
                    'max_depth':[1], 'n_estimators':[50]})

최상의 매개변수: {'colsample_bylevel': 0.9, 'colsample_bynode': 0.5, 'colsample_bytree': 0.8, 'gamma': 3, 'max_depth': 1, 'n_estimators': 50}
최상의 점수: 0.81874
